## Part 1

In [23]:
struct Maze
    m::Array{Char, 2}
end

Maze(s::Vector{T}) where T <: String = Maze(hcat(collect.(s)...))

find_element(m::Maze, el::Char) = get(CartesianIndices(m.m)[m.m .== el], 1, nothing)
find_elements(m::Maze, els) = [(x, find_element(m, x)) for x in els if find_element(m, x) != nothing]
mkeys(m::Maze) = Dict(find_elements(m, 'a':'z'))
doors(m::Maze) = Dict(find_elements(m, 'A':'Z'))
hero(m::Maze) = find_element(m, '@')

function neighbours(m::Maze, l)
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    [l + d for d in dirs[[m.m[l + d] != '#' for d in dirs]]]
end

function floodfill(m::Maze, start)
    arr = fill(-1, size(m.m)...)
    arr[start] = 0
    acc = [start]
    while !isempty(acc)
        loc = popfirst!(acc)
        for n in neighbours(m, loc)
            if arr[n] == -1 || arr[n] > arr[loc] + 1
                arr[n] = arr[loc] + 1
                push!(acc, n)
            end
        end
    end
    
    arr
end

function get_door_keys(maze::Maze, arr, start, finish)
    res = Char[]
    current = start
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    while current != finish
        for d in dirs
            if arr[current + d] + 1 == arr[current]
                current = current + d
                if maze.m[current] in collect('A':'Z') push!(res, lowercase(maze.m[current])) end
                break
            end
        end
    end
                
    res
end

function build_info(m::Maze)
    res = Dict()
    for (k, loc) in mkeys(m)
        arr = floodfill(m, loc)
        res[k] = Dict([(k1, (arr[v1], get_door_keys(m, arr, v1, loc))) for (k1, v1) in mkeys(m)])
    end
    
    arr = floodfill(m, hero(m))
    res['@'] = Dict([(k1, (arr[v1], get_door_keys(m, arr, v1, hero(m)))) for (k1, v1) in mkeys(m)])
    
    res
end

build_info (generic function with 1 method)

In [47]:
function part1(inp = "input.txt")
    m = Maze(readlines(inp))
    nkeys = length(mkeys(m)) + 1
    info = build_info(m)
    acc = [(['@'], 0)]
    res = []
    min_score = -1
    while !isempty(acc)
#        sort!(acc, by = x -> -x[2]/length(x[1]))
        
        ks, score = pop!(acc)
#        println(join(ks), " : ", score)
        if length(ks) == nkeys 
            if min_score < 0 || score < min_score min_score = score end
            println(join(ks), " : ", score)
            push!(res, (join(ks), score))
        end
        for (k, v) in info[ks[end]]
            if !(k in ks)
                if min_score > 0 && (score + v[1]) > min_score continue end
                if length(intersect(v[2], ks)) == length(v[2])
                    push!(acc, (vcat(ks, k), score + v[1]))
                end
            end
        end
    end
    
    return res
end

part1 (generic function with 2 methods)

In [49]:
m = Maze(readlines("test2.txt"))
println.(readlines("test2.txt"))
part1("test2.txt")

#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################
@bgcmaijhpekdofln : 176
@bgcmaijhpekdofnl : 174
@bgcmaijhpekdfonl : 174
@bgcmaijhpekdnofl : 174
@bgcmaijhpekdnfol : 174
@bgcmaijhpekfldno : 174
@bgcmaijhpekfdlno : 174
@bgcmaijhpekfnldo : 174
@bgcmaijhpekfndlo : 174
@bgcmaijhpekndofl : 174
@bgcmaijhpekndfol : 174
@bgcmaijhpeknfldo : 174
@bgcmaijhpeknfdlo : 174
@bgcmaijhpedkofnl : 174
@bgcmaijhpedkfonl : 174
@bgcmaijhpedknofl : 174
@bgcmaijhpedknfol : 174
@bgcmaijhpedofknl : 174
@bgcmaijhpedofnkl : 174
@bgcmaijhpedfoknl : 174
@bgcmaijhpedfonkl : 174
@bgcmaijhpednkofl : 174
@bgcmaijhpednkfol : 174
@bgcmaijhpednofkl : 174
@bgcmaijhpednfokl : 174
@bgcmaijhpefldkon : 172
@bgcmaijhpefldkno : 170
@bgcmaijhpefldnko : 170
@bgcmaijhpefkldno : 170
@bgcmaijhpefkdlno : 170
@bgcmaijhpefknldo : 170
@bgcmaijhpefkndlo : 170
@bgcmaijhpefdlkno : 170
@bgcmaijhpefdlnko : 170
@bgcmaijhpefnldko : 17

@bgcmaifhdloepjkn : 164
@bgcmaifhdloepjnk : 164
@bgcmaifhdlepjokn : 164
@bgcmaifhdlepjonk : 164
@bgcmaifhdlepojkn : 164
@bgcmaifhdlepojnk : 164
@bgcmaifhdlnpejok : 164
@bgcmaifhdlnpeojk : 164
@bgcmaifhdlnjpeok : 164
@bgcmaifhdlnjopek : 164
@bgcmaifhdlnjoepk : 164
@bgcmaifhdlnjepok : 164
@bgcmaifhdlnopejk : 164
@bgcmaifhdlnojpek : 164
@bgcmaifhdlnojepk : 164
@bgcmaifhdlnoepjk : 164
@bgcmaifhdlnepjok : 164
@bgcmaifhdlnepojk : 164
@bgcmaifdlhpejokn : 164
@bgcmaifdlhpejonk : 164
@bgcmaifdlhpeojkn : 164
@bgcmaifdlhpeojnk : 164
@bgcmaifdlhjpeokn : 164
@bgcmaifdlhjpeonk : 164
@bgcmaifdlhjopekn : 164
@bgcmaifdlhjopenk : 164
@bgcmaifdlhjoepkn : 164
@bgcmaifdlhjoepnk : 164
@bgcmaifdlhjepokn : 164
@bgcmaifdlhjeponk : 164
@bgcmaifdlhopejkn : 164
@bgcmaifdlhopejnk : 164
@bgcmaifdlhojpekn : 164
@bgcmaifdlhojpenk : 164
@bgcmaifdlhojepkn : 164
@bgcmaifdlhojepnk : 164
@bgcmaifdlhoepjkn : 164
@bgcmaifdlhoepjnk : 164
@bgcmaifdlhepjokn : 164
@bgcmaifdlhepjonk : 164
@bgcmaifdlhepojkn : 164
@bgcmaifdlhepojn

@bgcmahdpeiofjnkl : 158
@bgcmahdpeifojknl : 158
@bgcmahdpeifojnkl : 158
@bgcmahdipejofknl : 158
@bgcmahdipejofnkl : 158
@bgcmahdipejfoknl : 158
@bgcmahdipejfonkl : 158
@bgcmahdipeofjknl : 158
@bgcmahdipeofjnkl : 158
@bgcmahdipefojknl : 158
@bgcmahdipefojnkl : 158
@bgcmahdiepjofknl : 158
@bgcmahdiepjofnkl : 158
@bgcmahdiepjfoknl : 158
@bgcmahdiepjfonkl : 158
@bgcmahdiepofjknl : 158
@bgcmahdiepofjnkl : 158
@bgcmahdiepfojknl : 158
@bgcmahdiepfojnkl : 158
@bgcmahdjpeiofknl : 158
@bgcmahdjpeiofnkl : 158
@bgcmahdjpeifoknl : 158
@bgcmahdjpeifonkl : 158
@bgcmahdjipeofknl : 158
@bgcmahdjipeofnkl : 158
@bgcmahdjipefoknl : 158
@bgcmahdjipefonkl : 158
@bgcmahdjiepofknl : 158
@bgcmahdjiepofnkl : 158
@bgcmahdjiepfoknl : 158
@bgcmahdjiepfonkl : 158
@bgcmahdjofpeiknl : 158
@bgcmahdjofpeinkl : 158
@bgcmahdjofipeknl : 158
@bgcmahdjofipenkl : 158
@bgcmahdjofiepknl : 158
@bgcmahdjofiepnkl : 158
@bgcmahdjofepiknl : 158
@bgcmahdjofepinkl : 158
@bgcmahdjepiofknl : 158
@bgcmahdjepiofnkl : 158
@bgcmahdjepifokn

InterruptException: InterruptException:

In [3]:
arr = floodfill(m, CartesianIndex(18, 2))

24×5 Array{Int64,2}:
 -1  -1  -1  -1  -1
 -1  16  -1  28  -1
 -1  15  -1  27  -1
 -1  14  -1  26  -1
 -1  13  -1  25  -1
 -1  12  -1  24  -1
 -1  11  -1  23  -1
 -1  10  -1  22  -1
 -1   9  -1  21  -1
 -1   8  -1  20  -1
 -1   7  -1  19  -1
 -1   6  -1  18  -1
 -1   5  -1  17  -1
 -1   4  -1  16  -1
 -1   3  -1  15  -1
 -1   2  -1  14  -1
 -1   1  -1  13  -1
 -1   0  -1  12  -1
 -1   1  -1  11  -1
 -1   2  -1  10  -1
 -1   3  -1   9  -1
 -1   4  -1   8  -1
 -1   5   6   7  -1
 -1  -1  -1  -1  -1

In [4]:
get_door_keys(m, arr, CartesianIndex(2, 2), CartesianIndex(18, 2))

CartesianIndex(2, 2)CartesianIndex(18, 2)
16
0


4-element Array{Char,1}:
 'd'
 'e'
 'c'
 'a'

In [29]:
res = build_info(m)
keys(res['@'])

Base.KeySet for a Dict{Char,Tuple{Int64,Array{Char,1}}} with 6 entries. Keys:
  'f'
  'a'
  'c'
  'd'
  'e'
  'b'

In [28]:
intersect(res['@']['f'][2], ['a', 'c', 'x'])

2-element Array{Char,1}:
 'c'
 'a'

In [ ]:
struct Maze
    m::Array{Char, 2}
    score::Int
end

Maze(s::Vector{T}, score::Int) where T <: String = Maze(hcat(collect.(s)...), score)
Maze(s::Vector{T}) where T <: String = Maze(s, 0)

h(m::Maze, l1, l2) = sum(abs.(Tuple(l1 - l2)))

val(m::Maze, l) = m.m[l]

allowed(m::Maze) = [collect('a':'z'); '@'; '.']

upper(c::Char) = c + 'A' - 'a'

function neighbours(m::Maze, l)
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    [l + d for d in dirs[[val(m, l + d) in allowed(m) for d in dirs]]]
end

function neighbours2(m::Maze, l)
    dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
    [l + d for d in dirs[[val(m, l + d) != '#' for d in dirs]]]
end

function reconstruct_path(came_from::Dict{T, T}, current::T) where T
    path = T[]
    prev = current
    while current in keys(came_from)
        push!(path, current)
        current = came_from[current]
        prev = current
    end
    
    reverse(path)
end

function least(openset, scores)
    m = -1
    res = nothing
    for node in openset
        if m < 0 || scores[node][2] < m
            m = scores[node][2]
            res = node
        end
    end
    
    res
end

function astar(m::U, start::T, goal::T) where {T, U}
    openset = Set([start])
    scores = Dict(start => [0, h(m, start, goal)])
    came_from = Dict{T, T}()
    current = start
    while !isempty(openset)
        current = least(openset, scores)
        if current == goal
            return reconstruct_path(came_from, goal)
        end
        
        delete!(openset, current)
        for neighbour in neighbours(m, current)
            tentative_g_score = scores[current][1] + 1
            neighbour_g_score = get!(scores, neighbour, [-1, h(m, neighbour, goal)])[1]
            if neighbour_g_score < 0 || neighbour_g_score > tentative_g_score
                came_from[neighbour] = current
                scores[neighbour][1] = tentative_g_score
                if !(neighbour in openset) push!(openset, neighbour) end
            end
        end
    end
                
    if current != goal return T[] end
end

heur(hdict, c, ks) = sum([v for (k, v) in hdict[c] if k in ks])

In [ ]:
function floodfill(m::Maze)
    arr = fill(-1, size(m.m)...)
    start = hero(m)
    arr[start] = 0
    acc = [start]
    while !isempty(acc)
        loc = popfirst!(acc)
        for n in neighbours(m, loc)
            if arr[n] == -1 || arr[n] > arr[loc] + 1
                arr[n] = arr[loc] + 1
                push!(acc, n)
            end
        end
    end
    
    arr
end

function floodfill2(m::Maze, start)
    arr = fill(-1, size(m.m)...)
    arr[start] = 0
    acc = [start]
    while !isempty(acc)
        loc = popfirst!(acc)
        for n in neighbours2(m, loc)
            if arr[n] == -1 || arr[n] > arr[loc] + 1
                arr[n] = arr[loc] + 1
                push!(acc, n)
            end
        end
    end
    
    arr
end

function build_proto(m::Maze)
    res = Dict()
    for (k, loc) in mkeys(m)
        arr = floodfill2(m, loc)
        res[k] = Dict([(k1, arr[v1]) for (k1, v1) in mkeys(m)])
    end
    
    arr = floodfill2(m, hero(m))
    res['@'] = Dict([(k1, arr[v1]) for (k1, v1) in mkeys(m)])
    
    res
end

function least(openset, scores)
    m = -1
    res = nothing
    for node in openset
        if m < 0 || scores[node][2] < m
            m = scores[node][2]
            res = node
        end
    end
    
    res
end

function least2(openset, scores)
    m = -1
    res = nothing
    for node in openset
        if m < 0 || scores[node[2]] < m
            m = scores[node[2]]
            res = node
        end
    end
end

function xstar(m, start = "@")
    hdict = build_proto(m)
    openset = Set([(m, start)])
    scores = Dict(start => heur(hdict, start[end], keys(mkeys(m))))
    came_from = Dict()
    current = start
    while !isempty(openset)
        current = least2(openset, scores)
        current_maze, current_path, current_key = current
        
        if isempty(mkeys(current_maze))
             return (came_from, current_path, current_key)
        end
        
        delete!(openset, current)
        
        paths = floodfill(current_maze)
        available_keys = filter(x -> x[2] > 0, [(k, paths[v]) for (k, v) in mkeys(current_maze)])
        
        for (k, l) in available_keys
            new_m = copy(current_maze.m)
            new_m[hero(current_maze)] = '.'
            new_m[mkeys(current_maze)[k]] = '@'
            if uppercase(k) in keys(doors(current_maze)) new_m[doors(current_maze)[uppercase(k)]] = '.' end
            push!(openset, (Maze(new_m, state.score + l), current_path * k))
        end
        
        for (k, l) in available_keys
            tentative_g_score = scores[(current_path, current_key)][1] + l
            neighbour_g_score = get!(scores, (current_path*string(k), ))
        end
        
        for neighbour in neighbours(m, current)
            tentative_g_score = scores[current][1] + 1
            neighbour_g_score = get!(scores, neighbour, [-1, h(m, neighbour, goal)])[1]
            if neighbour_g_score < 0 || neighbour_g_score > tentative_g_score
                came_from[neighbour] = current
                scores[neighbour][1] = tentative_g_score
                if !(neighbour in openset) push!(openset, neighbour) end
            end
        end
    end
                
    if current != goal return T[] end
end

In [ ]:
find_element(m::Maze, el::Char) = get(CartesianIndices(m.m)[m.m .== el], 1, nothing)
find_elements(m::Maze, els) = [(x, find_element(m, x)) for x in els if find_element(m, x) != nothing]
mkeys(m::Maze) = Dict(find_elements(m, 'a':'z'))
doors(m::Maze) = Dict(find_elements(m, 'A':'Z'))
hero(m::Maze) = find_element(m, '@')

In [ ]:
m = Maze(readlines("input.txt"))
sum(values(build_proto(m)['u']))

In [ ]:
arr = floodfill(m)

In [ ]:
filter(x -> x[2] > 0, [(k, arr[v]) for (k, v) in mkeys(m)])
# [(k, arr[v]) for (k, v) in mkeys(m)]

In [ ]:
## A-star???

function part1(inp = "input.txt")
    m = Maze(readlines(inp))
    v1 = []
    acc = [(m, "")]
    min_score = -1
    while !isempty(acc)
        state, path = pop!(acc)
        println(join(path), " : ", state.score, " : ", min_score, " : ", length(v1))
        paths = floodfill(state)
        available_keys = filter(x -> x[2] > 0, [(k, paths[v]) for (k, v) in mkeys(state)])
        if isempty(available_keys)
            if isempty(mkeys(state))
                if min_score < 0 || state.score < min_score 
                    min_score = state.score 
                    push!(v1, (path, state.score))
                    continue
                end
            end
        end
        for (k, l) in available_keys
            if min_score > 0 && (state.score + l) > min_score continue end
            new_m = copy(state.m)
            new_m[hero(state)] = '.'
            new_m[mkeys(state)[k]] = '@'
            if uppercase(k) in keys(doors(state)) new_m[doors(state)[uppercase(k)]] = '.' end
            push!(acc, (Maze(new_m, state.score + l), path * k))
        end
    end
    
    v1
end

In [ ]:
h1(m::Maze, loc) = sum([sum(abs.(Tuple(loc - v))) for (k, v) in mkeys(m)])

In [ ]:
function part1(inp = "input.txt")
    m = Maze(readlines(inp))
    v1 = []
    acc = [(m, "")]
    min_score = -1
    while !isempty(acc)
        state, path = pop!(acc)
        println(join(path), " : ", state.score, " : ", min_score, " : ", length(v1))
        paths = floodfill(state)
        available_keys = filter(x -> x[2] > 0, [(k, paths[v]) for (k, v) in mkeys(state)])
        if isempty(available_keys)
            if isempty(mkeys(state))
                if min_score < 0 || state.score < min_score 
                    min_score = state.score 
                    push!(v1, (path, state.score))
                    continue
                end
            end
        end
        for (k, l) in available_keys
            if min_score > 0 && (state.score + l) > min_score continue end
            new_m = copy(state.m)
            new_m[hero(state)] = '.'
            new_m[mkeys(state)[k]] = '@'
            if uppercase(k) in keys(doors(state)) new_m[doors(state)[uppercase(k)]] = '.' end
            push!(acc, (Maze(new_m, state.score + l), path * k))
        end
    end
    
    v1
end

In [ ]:
res = part1("test2.txt")

In [ ]:
m = Maze(readlines("input.txt"))
hdict = build_proto(m)
heur(hdict, 'a', keys(mkeys(m)))
# 5806
# sum(values(build_proto(m)['@']))

In [ ]:
h1(m, hero(m))

In [ ]:
function f1(m)
#     mkeys(m)
#     hero(m)
    astar(m, hero(m), mkeys(m)['a'])
    # available_keys = filter( x -> x[2] > 0, [(k, length(astar(m, hero(m), loc))) for (k, loc) in mkeys(m)])
end

In [ ]:
using BenchmarkTools

@benchmark f1($m)

In [ ]:
mkeys(m)

In [ ]:
Profile.print()

In [ ]:
Profile.print(format=:flat)

In [ ]:
x0 = hero(m)
x1 = neighbours(m, hero(m))[4]
astar(m, x0, x1)

In [ ]:
ad = [(k, length(astar(m, hero(m), loc))) for (k, loc) in mkeys(m)]
ad = filter( x -> x[2] > 0, ad)

In [ ]:
find_element(m, 'a')

In [ ]:
m.m[find_element(m, 'a')] = '.'

In [ ]:
@benchmark mkeys($m)

In [ ]:
b = copy(a)

In [ ]:
b[2, 2] = 5

In [ ]:
a

In [ ]:
b

In [ ]:
dirs = CartesianIndex.([(0, 1), (1, 0), (0, -1), (-1, 0)])
#    l .+ dirs[val(m, l + d), allowed(m)) for d in dirs]

In [ ]:
dirs[[val(m, hero(m) + d) in allowed(m) for d in dirs]]

In [ ]:
2 in [3, 4, 2]

In [ ]:
paths = floodfill(m)
available_keys = filter(x -> x[2] > 0, [(k, paths[v]) for (k, v) in mkeys(m)])

In [ ]:
using BenchmarkTools

@benchmark neighbours($m, $hero(m))

In [ ]:
upper(c) = ('A' - 'a') + c

In [ ]:
upper('c')

In [ ]:
fill(-1, size(m.m)...)

In [ ]:
function ff(m)
    me = hero(m)
    available_keys = filter( x -> x[2] > 0, [(k, length(astar(m, hero(m), loc))) for (k, loc) in mkeys(state)])
end

In [ ]:
heur(hdict, c, ks) = sum([v for (k, v) in hdict[c] if k in ks])

In [ ]:
"abc"[end]